In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from basic_tools import *

debug=False

# Load variables

## Load phenotype information of individuals

In [2]:
phenotype_raw_AS=pd.read_csv(phenotype_raw_AS_path,sep='\t',index_col='ID')
phenotype_raw_AS['cohort']=1
phenotype_raw_CT=pd.read_csv(phenotype_raw_CT_path,sep='\t',index_col='ID')
phenotype_raw_CT['cohort']=2
phenotype_raw_NC=pd.read_csv(phenotype_raw_NC_path,sep='\t',index_col='ID')
phenotype_raw_NC.columns=phenotype_raw_NC.columns.str.replace('NCB','NC1')
phenotype_raw_NC['cohort']=3

phenotype_raw_AS.shape,phenotype_raw_CT.shape,phenotype_raw_NC.shape

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (9,11,158,552,1301) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (391,516) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (87,97,101,105,208,270,373,382,388) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


((7612, 1316), (99234, 578), (19004, 431))

In [3]:
phenotype_raw=pd.concat([phenotype_raw_AS,phenotype_raw_CT,phenotype_raw_NC])
phenotype_raw=phenotype_raw.sort_values('ID')
phenotype_raw.shape

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(125850, 2323)

In [4]:
#phenotype_raw=phenotype_raw.loc[['BHAN{:06d}'.format(i) for i in range(1,125872+1)]]

#assert sum(phenotype_raw.index.duplicated())==0
#len(phenotype_raw.index)
#phenotype_raw['CT1_GLU16_U']

## Load codebook

In [5]:
codebook_AS=pd.read_csv(codebook_AS_path)#,header=1)
codebook_CT=pd.read_csv(codebook_CT_path)#,header=1)
codebook_NC=pd.read_csv(codebook_NC_path)

codebook_column_dict={"테이블명(영문)":"table_code","변수명": "phenotype_code","변수설명":"description", "변수값(코드) 설명": "value_description","변수유형":"type","설문지":"questionnaire"}
codebook_AS=codebook_AS.rename(columns=codebook_column_dict)
codebook_CT=codebook_CT.rename(columns=codebook_column_dict)
codebook_NC=codebook_NC.rename(columns=codebook_column_dict)

codebook_AS['type']=codebook_AS['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')
codebook_CT['type']=codebook_CT['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')
codebook_NC['type']=codebook_NC['type'].replace('범주형','categorical').replace('연속형','continuous').replace('서술형','descriptive').replace('날짜','date')

codebook_AS['phenotype_code']=codebook_AS['phenotype_code'].str.upper()

codebook_total=pd.concat([codebook_AS,codebook_CT,codebook_NC])

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


# Run conversion manual

## syntax analyzer

In [6]:
def run_command(series_in,match_value,expression):
    series_out=series_in.copy()
    series_out[series_out.index]=np.nan
    if 'x' in match_value:
        #expression='(x>10)&(x!=77777)&(x!=99999)'
        #Only float variable is supported
        def match_value_function(x):
            try:
                return False if pd.isnull(x) else eval(match_value.replace('x',str(x))) 
            except:
                print(x,end='Error')
                return False
        match_bool=series_in.map(match_value_function)
        expression_function=lambda x:eval(expression.replace('x',str(x)))
        series_out[match_bool]=series_in[match_bool].map(expression_function)
    else:
        #print(np.sum(series_in==float(match_value)))
        try:
            match_value=float(match_value)
        except:
            pass
        try:
            expression=float(expression)
        except:
            pass
        #print(series_in.unique(),(series_in=='폐암').sum(),'폐암',match_value,match_value.strip("'")=='폐암',type('폐암'),type(match_value),match_value,expression)
        if type(match_value)==str and '+' in match_value:
            print("!! contain mode !!")
            series_out[series_in.map(lambda x: False if pd.isnull(x) or type(x)!=str else (match_value.replace('+','') in x))]=expression
        else:
            series_out[series_in==match_value]=expression
    return series_out

#conversion_manual.to_csv(conversion_manual_path,sep='\t',index=None)

## Run specific conversion (for debugging)

### Check if need

In [7]:
"""
(phenotype_raw['AS1_TRTAS']==2).sum(),(phenotype_raw['AS1_PDAS']==2).sum()
phenotype_raw.index[(phenotype_raw['CT1_ASTHCU']>=1) & (phenotype_raw['CT1_ASTHCU']<=4)].difference(phenotype_raw.index[phenotype_raw['CT1_ASTH']==2])
a=phenotype_raw.index[phenotype_raw['AS1_TRTAS']==2].difference(phenotype_raw.index[phenotype_raw['AS1_PDAS']==1])
len(a)

phenotype_raw['NC1_CA1'].unique(),phenotype_raw['NC1_CA_NA2'].unique()

phenotype_raw['AS1_HIP1'].hist()

(phenotype_raw['AS1_HIP1']-phenotype_raw['AS1_HIP2']).hist()

phenotype_raw_CT.columns[phenotype_raw_CT.columns.str.contains('BLOOD')]
(phenotype_raw['NC1_CRP'][phenotype_raw['NC1_CRP']!=99999]*0.1).median()
(phenotype_raw['AS1_PDDEM']).value_counts()
#(phenotype_raw['AS1_PREGNO']-phenotype_raw['AS1_CHILD_P']-phenotype_raw['AS1_CHILD_D']).value_counts()
"""

"\n(phenotype_raw['AS1_TRTAS']==2).sum(),(phenotype_raw['AS1_PDAS']==2).sum()\nphenotype_raw.index[(phenotype_raw['CT1_ASTHCU']>=1) & (phenotype_raw['CT1_ASTHCU']<=4)].difference(phenotype_raw.index[phenotype_raw['CT1_ASTH']==2])\na=phenotype_raw.index[phenotype_raw['AS1_TRTAS']==2].difference(phenotype_raw.index[phenotype_raw['AS1_PDAS']==1])\nlen(a)\n\nphenotype_raw['NC1_CA1'].unique(),phenotype_raw['NC1_CA_NA2'].unique()\n\nphenotype_raw['AS1_HIP1'].hist()\n\n(phenotype_raw['AS1_HIP1']-phenotype_raw['AS1_HIP2']).hist()\n\nphenotype_raw_CT.columns[phenotype_raw_CT.columns.str.contains('BLOOD')]\n(phenotype_raw['NC1_CRP'][phenotype_raw['NC1_CRP']!=99999]*0.1).median()\n(phenotype_raw['AS1_PDDEM']).value_counts()\n#(phenotype_raw['AS1_PREGNO']-phenotype_raw['AS1_CHILD_P']-phenotype_raw['AS1_CHILD_D']).value_counts()\n"

In [8]:
name_filter='glucose_in_blood'
test_match_value=1
test_conversion_value=2

conversion_manual=pd.read_csv(conversion_manual_path,sep='\t',index_col='phenotype_code')

##Run all 'commands' with name=='new_name'
conversion_list_dict={}
for idx,row in conversion_manual[~pd.isnull(conversion_manual['commands'])].iterrows():
    for command in row['commands'].split('|'):
        new_name,match_value,expression=command.split('@')
        if name_filter!=None and new_name!=name_filter:
            continue
        print(idx,command,phenotype_raw[idx].unique())
        print("# of filtered={} before: ".format(test_match_value),(phenotype_raw[idx]==test_match_value).sum())
        conversion=run_command(phenotype_raw[idx],match_value,expression)
        conversion_list_dict[new_name]=conversion_list_dict.get(new_name,[])+[conversion]
        print("# of filtered={} after: ".format(test_conversion_value),(conversion==test_conversion_value).sum(),", conversion unique",conversion.unique())
        print('-----------------------------------------')

In [9]:
print("--------------Conversion----------------------")
# merge results of same 'new_name'
conversion_dict={}
for new_name,conversion_list in conversion_list_dict.items():
    merge_result=None
    for conversion_result in conversion_list:
        if merge_result is None:
            merge_result= conversion_result
        else:
            merge_result.update(conversion_result)
    print(new_name,str(merge_result.unique())[:30])
    conversion_dict[new_name]=merge_result
#cohort 별 분포
print('# of null:', conversion_dict[name_filter].isnull().sum())
conversion_dict[name_filter].value_counts()

--------------Conversion----------------------


KeyError: 'glucose_in_blood'

In [ ]:
print('# of {} in AS:'.format(test_conversion_value),\
      (phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==1).sum(),'/',\
      (phenotype_raw['cohort']==1).sum(),\
      '{:.2f}'.format((phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==1).sum()/(phenotype_raw['cohort']==1).sum()*100),\
     )
print('# of {} in CT:'.format(test_conversion_value),\
      (phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==2).sum(),'/',\
      (phenotype_raw['cohort']==2).sum(),\
      '{:.2f}'.format((phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==2).sum()/(phenotype_raw['cohort']==2).sum()*100),\
     )

print('# of {} in NC:'.format(test_conversion_value),\
      (phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==3).sum(),'/',\
      (phenotype_raw['cohort']==3).sum(),\
      '{:.2f}'.format((phenotype_raw[conversion_dict[name_filter]==test_conversion_value]['cohort']==3).sum()/(phenotype_raw['cohort']==3).sum()*100),\
     )

      #'---','{:.2f}'.format(phenotype_raw['AS1_contain'][conversion_dict[name_filter]==test_conversion_value].sum()/phenotype_raw['AS1_contain'].sum()*100))

In [ ]:
conversion_dict[name_filter].hist()
plt.show()

## Run all conversion (for debugging)

In [8]:
conversion_manual=pd.read_csv(conversion_manual_path,sep='\t',index_col='phenotype_code')

In [8]:
#conversion_list_dict.keys()

In [10]:
name_filter=None
test_match_value=1
test_conversion_value=2

##Run all 'commands' with name=='new_name'
conversion_list_dict={}
for idx,row in conversion_manual[~pd.isnull(conversion_manual['commands'])].iterrows():
    for command in row['commands'].split('|'):
        new_name,match_value,expression=command.split('@')
        if name_filter!=None and new_name!=name_filter:
            continue
        print(idx,command,phenotype_raw[idx].unique())
        print("# of filtered={} before: ".format(test_match_value),(phenotype_raw[idx]==test_match_value).sum())
        conversion=run_command(phenotype_raw[idx],match_value,expression)
        conversion_list_dict[new_name]=conversion_list_dict.get(new_name,[])+[conversion]
        print("# of filtered={} after: ".format(test_conversion_value),(conversion==test_conversion_value).sum(),", conversion unique",conversion.unique())
        print('-----------------------------------------')

NC1_AGE age@(x!=66666)&(x!=77777)&(x!=99999)@x [67. 60. 51. 70. 68. 77. 65. 50. 56. 74. 62. 72. 69. 49. 57. 59. 53. 52.
 55. 73. 78. 71. 45. 61. 75. 63. 46. 54. 80. 66. 64. 47. 44. 58. 48. 76.
 79. 41. 86. 87. 40. 81. 43. 82. 42. 85. 84. 90. 83. 88. nan]
# of filtered=1 before:  0
# of filtered=2 after:  0 , conversion unique [67. 60. 51. 70. 68. 77. 65. 50. 56. 74. 62. 72. 69. 49. 57. 59. 53. 52.
 55. 73. 78. 71. 45. 61. 75. 63. 46. 54. 80. 66. 64. 47. 44. 58. 48. 76.
 79. 41. 86. 87. 40. 81. 43. 82. 42. 85. 84. 90. 83. 88. nan]
-----------------------------------------
CT1_AGE age@(x!=66666)&(x!=77777)&(x!=99999)@x [nan 63. 45. 59. 50. 51. 61. 55. 48. 53. 60. 70. 57. 66. 44. 46. 56. 58.
 52. 41. 67. 47. 40. 42. 49. 43. 54. 62. 65. 69. 64. 68. 71. 72. 73. 76.
 75. 74. 77. 79. 78.]
# of filtered=1 before:  0
# of filtered=2 after:  0 , conversion unique [nan 63. 45. 59. 50. 51. 61. 55. 48. 53. 60. 70. 57. 66. 44. 46. 56. 58.
 52. 41. 67. 47. 40. 42. 49. 43. 54. 62. 65. 69. 64. 68. 71. 

In [11]:
print("--------------Conversion----------------------")
# merge results of same 'new_name'
conversion_dict={}
for new_name,conversion_list in conversion_list_dict.items():
    merge_result=None
    for conversion_result in conversion_list:
        if merge_result is None:
            merge_result= conversion_result
        else:
            merge_result.update(conversion_result)
    print(new_name,str(merge_result.unique())[:30])
    conversion_dict[new_name]=merge_result
#cohort 별 분포

--------------Conversion----------------------
age [67. 60. 51. 70. 68. 77. 65. 5
sex [2. 1.]
asthma [nan  2.]
chronic_bronchitis [nan  2.]
allergic_disease [nan  2.]
stomach_cancer [nan 2.0]
liver_cancer [nan 2.0]
colorectal_cancer [nan 2.0]
breast_cancer [nan 2.0]
cervical_cancer [nan 2.0]
lung_cancer [nan 2.0]
thyroid_cancer [nan 2.0]
prostate_cancer [nan 2.0]
bladder_cancer [nan 2.0]
colon_polyps [nan  2.]
angina_pectoris_myocardial_infarction [nan  2.]
hypertension [nan  2.]
stroke [nan  2.]
transient_ischemic_attacks [nan  2.]
acute_liver_disease [nan  2.]
diabetes [ 2. nan]
t2_diabetes [ 2. nan]
hyperlipidemia [nan  2.]
thyroid_disease [nan  2.]
arthritis [nan  2.]
rheumatoid_arthritis [nan  2.]
osteoporosis [nan  2.]
parkinson [nan  2.]
pwi_total [nan 27.  7. 10. 11. 12. 37.  
chronic_gastritis_stomach_ulcer [nan  2.]
duodenal_ulcer [nan  2.]
urinary_tract_infection [nan  2.]
height [136.7 159.4 162.5 143.3 154. 
hip_circumference [ 92.    94.4   85.6   94.5   
waist_circumfere

## converting to Pandas DataFrame

In [12]:
conversion_df=pd.DataFrame(conversion_dict)
conversion_df['cohort']=phenotype_raw['cohort']
conversion_df['bmi']=(conversion_df['weight']/((0.01*conversion_df['height'])**2)).round(2)

## (optional) human-readable transformation manual

In [13]:
conversion_manual_nonull=conversion_manual[~pd.isnull(conversion_manual['commands'])]
conversion_manual_nonull['new_name']=conversion_manual_nonull['commands'].map(lambda x: x.split('|')[0].split('@')[0])

/home/ch6845/tools/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
#conversion_manual_nonull

In [15]:
conversion_manual_codebook=pd.DataFrame(columns=pd.MultiIndex.from_product([[i for i in range(0,15)], list(codebook_column_dict.values())+['commands']], names=['code', 'data']),index=conversion_manual_nonull.new_name.unique())

In [16]:
for new_name in conversion_manual_nonull.new_name.unique():
    conversion_manual_nonull_select=conversion_manual_nonull[conversion_manual_nonull['new_name']==new_name]
    for idx,(phenotype_code,row) in enumerate(conversion_manual_nonull_select.iterrows()):
        codebook_total_select=codebook_total[codebook_total['phenotype_code']==phenotype_code][codebook_column_dict.values()].iloc[0]
        codebook_total_select['commands']=row['commands']
        conversion_manual_codebook.loc[row['new_name']][idx]=codebook_total_select

In [17]:
#conversion_manual_codebook

In [18]:
conversion_manual_codebook.to_csv(conversion_manual_codebook_path,encoding='utf8')

# Clean up

## Saving Phenotype

In [34]:
conversion_df=pd.DataFrame(conversion_dict)
conversion_df['cohort']=phenotype_raw['cohort']
conversion_df['bmi']=(conversion_df['weight']/((0.01*conversion_df['height'])**2)).round(2)

In [35]:
np.unique(phenotype_raw['cohort'].values,return_counts=True)

(array([1, 2, 3]), array([ 7612, 99234, 19004]))

In [36]:
try_float_to_int=lambda x: x if np.isnan(x) else x if x%1!=0 else np.int(x)

#### all

In [37]:
conversion_df_try_int=conversion_df.applymap(try_float_to_int)

In [38]:
conversion_df_try_int.to_csv(pheno_all_file_path,sep='\t')

In [39]:
#conversion_df_try_int['rheumatoid_arthritis']
pheno_all_file_path

'data/phenotype/phenotypes_all.tsv'

In [40]:
conversion_df_try_int.shape

(125850, 101)

In [41]:
#conversion_df_try_int[conversion_df_try_int.columns.str.contains('arth')]

In [42]:
(conversion_df_try_int['blood_in_urine']==2).sum()

1822